In [3]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

transformations can be applied either in the __init__ method or in the __getitem__ method of your WineDataset class, depending on your specific requirements and the nature of the transformations.

- Apply transformations in __init__: If  transformations are applied to the entire dataset and do not vary between different samples, we can apply them in the __init__ method. This approach is suitable for transformations such as normalization, resizing, or data augmentation.

- Apply transformations in __getitem__: If transformations need to be applied differently to each sample or if they depend on the index or any other sample-specific information, we should apply them in the __getitem__ method. This approach is useful for transformations that involve randomization or require access to individual samples.

In [26]:
class WineDataset(Dataset):
    def __init__(self, transform = None):
        xy = np.loadtxt("./wine.csv", delimiter=",", dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        self.x = xy[:, 1:]
        self.y = xy[:, [0]]
        # transform
        self.transform = transform
    
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def __len__(self):
        return self.n_samples

In [27]:
class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample
        return torch.from_numpy(inputs), torch.from_numpy(labels)

In [31]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, sample):
        inputs, labels = sample
        inputs *= self.factor
        return inputs, labels

In [28]:
dataset = WineDataset(transform = ToTensor())

In [29]:
item = dataset[0]
features, labels = item
features, labels

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor([1.]))

In [30]:
print(type(features))

<class 'torch.Tensor'>


# trying MulTransform

In [32]:
dataset_mul_transform = WineDataset(transform = MulTransform(2))

In [34]:
dataset_mul_transform[0]

(array([2.846e+01, 3.420e+00, 4.860e+00, 3.120e+01, 2.540e+02, 5.600e+00,
        6.120e+00, 5.600e-01, 4.580e+00, 1.128e+01, 2.080e+00, 7.840e+00,
        2.130e+03], dtype=float32),
 array([1.], dtype=float32))

In [36]:
dataset[0]

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor([1.]))

# composed tranforoms

In [38]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset_composed = WineDataset(transform = composed)

In [40]:
item = dataset_composed[0]
features, labels = item
features, labels

(tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
         6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
         2.1300e+03]),
 tensor([1.]))